In [ ]:
! pip install kss konlpy
import nltk, kss, konlpy
from konlpy.tag import Okt # 한국어 단어 토크나이저

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:

t_data = '나는 오늘은 습기가 높아서 슬퍼 강의는 잘 듣고 있니?'
tk = Okt()

In [ ]:
def 문장_처리(in_t):
    입력문자 = in_t.replace('?','')
    토큰_data= tk.morphs(입력문자) #모프,전체부르기?      
    단어집합 = {}    #보캐만들기
    단어빈도수 = []
    for 단어 in 토큰_data:
        if 단어 not in 단어집합 : #단어집합 딕셔너리에 없다면 # 딕셔너리에키값으로
            단어집합[단어] = len(단어집합)  # 키와밸류 추가
            단어빈도수.insert(len(단어집합)-1 , 1) # insert    # 1:처음들어가니까
        else:
            단어인덱스 = 단어집합.get(단어)  # 단어의 인덱스 # get 밸류뽑아오는
            단어빈도수[단어인덱스] += 1 #횟수 기록
    return 단어집합, 단어빈도수 #튜플로 두개 리턴

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
t_t = t_data.replace('?','')
bv = CountVectorizer()
bv.fit_transform([t_t]).toarray()

array([[1, 1, 1, 1, 1, 1, 1, 1]])

In [ ]:
문서 = [ '나는 강의는 어려워', '나는 날씨가 덥고 습기가 많아', '나는 오늘은 그냥 있니?' , '나는 수업은 듣고 있니?' ]

In [ ]:
bv.vocabulary_

{'강의는': 0, '나는': 1, '높아서': 2, '듣고': 3, '슬퍼': 4, '습기가': 5, '오늘은': 6, '있니': 7}

In [ ]:
# DTM 행렬 / BOW빈도
bv.transform(문서).toarray()

array([[1, 1, 0, 0, 0, 0, 0, 0],
       [0, 1, 0, 0, 0, 1, 0, 0],
       [0, 1, 0, 0, 0, 0, 1, 1],
       [0, 1, 0, 1, 0, 0, 0, 1]])

In [ ]:
bv2 = CountVectorizer(stop_words=['나는']) #불용어=나는
bv2.fit_transform(문서).toarray()

array([[1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0],
       [0, 0, 1, 1, 0, 1, 0, 1, 0, 0, 0],
       [0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1],
       [0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1]])

In [ ]:
bv2.vocabulary_ # '나는'이 없다

{'강의는': 0,
 '그냥': 1,
 '날씨가': 2,
 '덥고': 3,
 '듣고': 4,
 '많아': 5,
 '수업은': 6,
 '습기가': 7,
 '어려워': 8,
 '오늘은': 9,
 '있니': 10}

TF-IDF    #단어마다 가중치를 다르게 주는 것

In [ ]:
t_data = [ 
    '공부 하고 싶다' ,
    '공부 하고 싫다' ,
    '공부 내일 할까' ,
    '공부 하고 놀기' ,
    '놀기 말고 놀자' ] 

from math import log
import pandas as pd 


In [ ]:
vos = list(set( 단어 for 문장 in t_data for 단어 in 문장.split())) # 중복방지를위한set
vos

['할까', '하고', '내일', '싫다', '공부', '놀기', '말고', '놀자', '싶다']

In [ ]:
type(vos),type(t_data)

(list, list)

In [ ]:
len(vos), len(t_data)

(9, 5)

In [ ]:
import pandas as pd
pd.DataFrame(t_data)

,0
0,공부 하고 싶다
1,공부 하고 싫다
2,공부 내일 할까
3,공부 하고 놀기
4,놀기 말고 놀자


In [ ]:
pd.DataFrame(vos)

,0
0,할까
1,하고
2,내일
3,싫다
4,공부
5,놀기
6,말고
7,놀자
8,싶다


In [ ]:
# TF-IDF  공식 코드화
n=len(t_data)
def tf(t,d):                #  tf는 몇번 나왔는지 카운트
    return d.count(t)
def idf(t):
    df=0 #횟수
    for 문장 in t_data:
        df +=  t in 문장   # 문장안에 t 가 있다면 1이 df에 전달됨
    return log(n/(df+1))                   # idf 공식    idf(d,t) = log ( n / 1+df(t) )
def tfidf(t,d):
    return tf(t,d)*idf(t)

In [ ]:
# DTM = tf table
t_l = []
for i in range(n): #문서 수 n개
    t_l.append([]) # 이차원리스트를만들기위한  #인덱스넣을칸작업
    d = t_data[i]
    for j in range(len(vos)): #vos 보캐불러셋집합
        t = vos[j]
        t_l[-1].append(tf(t,d))
import pandas as pd
tf_ = pd.DataFrame(t_l, columns = vos)
tf_  # 싶다 - 1이 하나뿐 > 문장에서 주요의미 > 가중치 높게

,싫다,놀기,공부,할까,하고,싶다,내일,말고,놀자
0,0,0,1,0,1,1,0,0,0
1,1,0,1,0,1,0,0,0,0
2,0,0,1,1,0,0,1,0,0
3,0,1,1,0,1,0,0,0,0
4,0,1,0,0,0,0,0,1,1


In [ ]:
idf_l = []
for i in range(len(vos)):
    t=vos[i]
    idf_l.append(idf(t))
idf_ = pd.DataFrame(idf_l , index=vos , columns=['IDF']) 
idf_
# 각단어별 idf  가중치를 보여준다  단어마다 weight 곱셈 = 벡터화

,IDF
싶다,0.916291
공부,0.000000
하고,0.223144
할까,0.916291
놀기,0.510826
싫다,0.916291
내일,0.916291
놀자,0.916291
말고,0.916291


In [ ]:
tf_idf_l = []
for i in range(n):
    tf_idf_l.append([])
    d = t_data[i]
    for j in range(len(vos)):
        t = vos[j]
        tf_idf_l[-1].append(tfidf(t,d))
tf_idf_ = pd.DataFrame(tf_idf_l, columns=vos )
tf_idf_             
# 데이터테이블: tf 와 idf 곱   tf-idf

,싶다,공부,하고,할까,놀기,싫다,내일,놀자,말고
0,0.916291,0.0,0.223144,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,0.000000,0.0,0.223144,0.000000,0.000000,0.916291,0.000000,0.000000,0.000000
2,0.000000,0.0,0.000000,0.916291,0.000000,0.000000,0.916291,0.000000,0.000000
3,0.000000,0.0,0.223144,0.000000,0.510826,0.000000,0.000000,0.000000,0.000000
4,0.000000,0.0,0.000000,0.000000,0.510826,0.000000,0.000000,0.916291,0.916291


In [ ]:
# 국소표현.이산표현  BOW DTM  TF-IDF  (임베딩도 일종의 국소표현 임베딩을 하면 된다 단 유사도검사시 TF IDF로)
# 분산표현.연속표현  Word2Vec, FastText

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
t_data 

['공부 하고 싶다', '공부 하고 싫다', '공부 내일 할까', '공부 하고 놀기', '놀기 말고 놀자']

In [ ]:
# tf idf 라이브러리

In [ ]:
tfidfv = TfidfVectorizer().fit(t_data)
tfidfv.vocabulary_

{'공부': 0,
 '내일': 1,
 '놀기': 2,
 '놀자': 3,
 '말고': 4,
 '싫다': 5,
 '싶다': 6,
 '하고': 7,
 '할까': 8}

In [ ]:
tfidfv.transform(t_data).toarray()

array([[0.42395393, 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.75251519, 0.5039682 , 0.        ],
       [0.42395393, 0.        , 0.        , 0.        , 0.        ,
        0.75251519, 0.        , 0.5039682 , 0.        ],
       [0.37008621, 0.65690037, 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.65690037],
       [0.47330879, 0.        , 0.6778033 , 0.        , 0.        ,
        0.        , 0.        , 0.56263796, 0.        ],
       [0.        , 0.        , 0.49552379, 0.61418897, 0.61418897,
        0.        , 0.        , 0.        , 0.        ]])

In [ ]:
# tfidfv.transform(t_data).toarray().shape # 문장당9차원 9피처    > 원핫 9*9차원

(5, 9)

In [ ]:
import numpy as np
#벡터데이터 = 1차원어레이 = tensor
data1 = np.array([1,1,1,1]) # 데이터관점에서는 4차원이다 데이터관점에서는 피처수가 차원
np.linalg.norm(data1) # linalg.norm  벡터를 정규화  # 원점에서 점으로 선 긋기

2.0

In [ ]:
def cos_i(A,B): #코사인유사도 # 벡터간기울기비교 방향성이 같은지 유사도 판별  #방향같으면 1 역방향이면 -1   구십도면 0 (긍정방향/부정방향)
    return A.dot(B)/(np.linalg.norm(A)*np.linalg.norm(B))

In [ ]:
d1 = np.array( [0,5,5,5])
d2 = np.array( [0,-1,-1,-1])
cos_i(d1,d2)

-1.0

In [ ]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer  
from sklearn.metrics.pairwise import cosine_similarity


In [ ]:
data = pd.read_csv('data1.csv', low_memory=False)


In [ ]:
data = data.head(20000)
data['overview'] = data['overview'].fillna('')

In [ ]:
tfidf = TfidfVectorizer (stop_words='english') #문장가져오기
tfidf_t = tfidf.fit_transform(data['overview']) 
tfidf_t

<20000x47487 sparse matrix of type '<class 'numpy.float64'>'
	with 535909 stored elements in Compressed Sparse Row format>

In [ ]:

cos = cosine_similarity(tfidf_t, tfidf_t) # 이만*이만 값
cos.shape

(20000, 20000)

In [ ]:
data.index

RangeIndex(start=0, stop=20000, step=1)

In [ ]:
t_idx = dict(zip(data['title'], data.index ))
t_idx

{'Toy Story': 0,
 'Jumanji': 1,
 'Grumpier Old Men': 2,
 'Waiting to Exhale': 3,
 'Father of the Bride Part II': 4,
 'Heat': 14684,
 'Sabrina': 888,
 'Tom and Huck': 7,
 'Sudden Death': 8,
 'GoldenEye': 9,
 'The American President': 10,
 'Dracula: Dead and Loving It': 11,
 'Balto': 12,
 'Nixon': 13,
 'Cutthroat Island': 14,
 'Casino': 15,
 'Sense and Sensibility': 16,
 'Four Rooms': 17,
 'Ace Ventura: When Nature Calls': 18,
 'Money Train': 19,
 'Get Shorty': 20,
 'Copycat': 21,
 'Assassins': 22,
 'Powder': 23,
 'Leaving Las Vegas': 24,
 'Othello': 15347,
 'Now and Then': 26,
 'Persuasion': 14838,
 'The City of Lost Children': 28,
 'Shanghai Triad': 29,
 'Dangerous Minds': 30,
 'Twelve Monkeys': 31,
 'Wings of Courage': 32,
 'Babe': 33,
 'Carrington': 34,
 'Dead Man Walking': 35,
 'Across the Sea of Time': 36,
 'It Takes Two': 37,
 'Clueless': 38,
 'Cry, the Beloved Country': 39,
 'Richard III': 17719,
 'Dead Presidents': 41,
 'Restoration': 42,
 'Mortal Kombat': 43,
 'To Die For': 44,

In [ ]:
def ck_s_t(t, cosine_sim = cos ):
    idx = t_idx[t]
    c_sc = list(enumerate(cosine_sim[idx])) # 넘버링 #이누머-열거
    c_sc = sorted ( c_sc, key=lambda a:a[1] , reverse=True)
    return c_sc[0]


In [ ]:
ck_s_t('Toy Story')


(0, 1.0)

In [ ]:
def ck_s_t(t, cosine_sim = cos ):
    idx = t_idx[t]
    c_sc = list(enumerate(cosine_sim[idx])) # 넘버링 #이누머-열거
    c_sc = sorted ( c_sc, key=lambda a:a[1] , reverse=True)
    return c_sc[1] 
# [1] 유사도

In [ ]:
ck_s_t('Toy Story')


(15348, 0.5258229300737997)

In [ ]:
def ck_s_t(t, cosine_sim = cos ):
    idx = t_idx[t]
    c_sc = list(enumerate(cosine_sim[idx])) # 넘버링 #이누머-열거
    c_sc = sorted ( c_sc, key=lambda a:a[1] , reverse=True)
    m_d = c_sc[1:6] 
    m_i = [i[0]for i in m_d]
    return data['title'].iloc[m_i]
    

In [ ]:
ck_s_t('Toy Story')


15348               Toy Story 3
2997                Toy Story 2
10301    The 40 Year Old Virgin
8327                  The Champ
1071      Rebel Without a Cause
Name: title, dtype: object

In [ ]:
#자카드유사도

In [ ]:
A = set([1,2,3,4,5])
B = set([4,5,6,7,8,9])
A|B  # or   합집합

{1, 2, 3, 4, 5, 6, 7, 8, 9}

In [ ]:
A&B #and 교집합

{4, 5}

In [ ]:
data1 = '나는 오늘 정말 피곤해 그래서 힘들어'.split()
data2 = '나는 오늘 정말 못해 그래서 힘들어'.split()
data3 = '나는 오늘 너무 즐거와 그래서 행복해'.split()


In [ ]:
data1

['나는', '오늘', '정말', '피곤해', '그래서', '힘들어']

In [ ]:
un=set(data1) | set(data2)
un

{'그래서', '나는', '못해', '오늘', '정말', '피곤해', '힘들어'}

In [ ]:
intd=set(data1) & set(data2)
intd

{'그래서', '나는', '오늘', '정말', '힘들어'}

In [ ]:
len(intd)/len(un) #이것이 유사도

0.7142857142857143

In [ ]:
un1=set(data1) | set(data3)
intd1=set(data1) & set(data3)
len(intd1)/len(un1) #이것이 유사도   #data1은 data2와 더 유사도가 높다

0.3333333333333333

In [ ]:
data1 = '나는 오늘 정말 피곤해 그래서 힘들어' 
data2 = '나는 오늘 정말 못해 그래서 힘들어' 
data3 = '나는 오늘 너무 즐거와 그래서 행복해' 


In [ ]:
t_v = TfidfVectorizer()
m_data = t_v.fit_transform( [data1,data2,data3])
m_data.toarray()
#3개의 문장이 벡터화 점 찍혓다

array([[0.32999531, 0.32999531, 0.        , 0.        , 0.32999531,
        0.42492904, 0.        , 0.55873062, 0.        , 0.42492904],
       [0.32999531, 0.32999531, 0.        , 0.55873062, 0.32999531,
        0.42492904, 0.        , 0.        , 0.        , 0.42492904],
       [0.29360705, 0.29360705, 0.49711994, 0.        , 0.29360705,
        0.        , 0.49711994, 0.        , 0.49711994, 0.        ]])

In [ ]:
m_data.toarray().shape # 단어10개 피처10개 해당피처에 숫자값

(3, 10)

In [ ]:
#유클리드유사도   점과점사이의거리 (피타고라스정리활용)

In [ ]:
def f(A,B):
    return np.sqrt( np.sum((A-B)**2)  )
    

In [ ]:
m_data[0].toarray()

array([[0.32999531, 0.32999531, 0.        , 0.        , 0.32999531,
        0.42492904, 0.        , 0.55873062, 0.        , 0.42492904]])

In [ ]:
m_data.toarray()[0]

array([0.32999531, 0.32999531, 0.        , 0.        , 0.32999531,
       0.42492904, 0.        , 0.55873062, 0.        , 0.42492904])

In [ ]:
A = m_data.toarray()[0]
B = m_data.toarray()[1]
C = m_data.toarray()[2]

In [ ]:
f(A,B), f(A,C)

(0.7901644267844229, 1.1910777960950294)

In [ ]:
def e_f(m):
    return m/np.sum(m)
e_m_d = e_f(m_data) #정규화끝난매트릭스데이터

In [ ]:
from sklearn.metrics import euclidean_distances #유클리드거리 소문자-매소드
euclidean_distances(m_data[0:1], m_data[1:2]) #거리값

array([[0.79016443]])

In [ ]:
euclidean_distances(e_m_d[0:1], e_m_d[1:2])

array([[0.11021454]])

In [ ]:
euclidean_distances(e_m_d[0:1], e_m_d[2:])

array([[0.16613516]])

In [ ]:
# 멘하탄유사도    

In [ ]:
from sklearn.metrics.pairwise import manhattan_distances
print( manhattan_distances(m_data[0:1], m_data[1:2])  , manhattan_distances(m_data[0:1], m_data[2:])  )
print( manhattan_distances(e_m_d[0:1], e_m_d[1:2])  ,manhattan_distances(m_data[0:1], m_data[2:])     )

[[1.11746125]] [[3.00911332]]
[[0.1558669]] [[3.00911332]]


In [ ]:

맨하탄유사도 :사각형격자로이루어진지도에서출발에서도착까지가로지르지않고라수잇는최단거리
코사인유사도 두벡터사이의코사인각도를구하는방법 -1~1  1에가까울수록유사함
              가장 널리쓰임   코사인세타=  a백dot b백/ a b  코드자체가 코사인세타
유클리디언유사도  엔차원 공간의 두 점 사이의 최단거리
자카드유사도  두문장을각단어의집합으로만든뒤집합을통해유사도르측정 일에가까울수록유사도가높음 영부터일사이의값을가짐



In [ ]:
data = [data1,data2,data3]
from tensorflow.keras.preprocessing.text import Tokenizer
tk = Tokenizer()
tk.fit_on_texts(data)
tk.index_word

NameError: ignored

In [ ]:
tk.texts_to_matrix(data, mode='count')

array([[0., 1., 1., 1., 1., 1., 1., 0., 0., 0., 0.],
       [0., 1., 1., 1., 1., 1., 0., 1., 0., 0., 0.],
       [0., 1., 1., 1., 0., 0., 0., 0., 1., 1., 1.]])

In [ ]:
tk.texts_to_matrix(data, mode='tfidf')

array([[0.        , 0.55961579, 0.55961579, 0.55961579, 0.69314718,
        0.69314718, 0.91629073, 0.        , 0.        , 0.        ,
        0.        ],
       [0.        , 0.55961579, 0.55961579, 0.55961579, 0.69314718,
        0.69314718, 0.        , 0.91629073, 0.        , 0.        ,
        0.        ],
       [0.        , 0.55961579, 0.55961579, 0.55961579, 0.        ,
        0.        , 0.        , 0.        , 0.91629073, 0.91629073,
        0.91629073]])

In [ ]:
from tensorflow.keras.preprocessing.text import Tokenizer

In [ ]:
from sklearn.datasets import fetch_20newsgroups #분류 문제 풀기
import pandas as pd
import numpy as np
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.utils import to_categorical  
import matplotlib.pyplot as plt

In [ ]:
w_data = fetch_20newsgroups()   #딕셔너리? 데이터 로드
w_data.keys()

dict_keys(['data', 'filenames', 'target_names', 'target', 'DESCR'])

In [ ]:
X_data = w_data.data #data 시리즈슬라이스
y_data = w_data.target #타겟시리즈슬라이스

In [ ]:
len(X_data), len(y_data) #txt라이상치확인어렵다
#


(11314, 11314)

In [ ]:
len(np.unique(X_data)), np.unique(y_data)

(11314,
 array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
        17, 18, 19]))

In [ ]:
all_data = pd.DataFrame()
all_data['X']=X_data
all_data['y']=y_data
all_data

,X,y
0,From: lerxst@wam.umd.edu (where's my thing)\nS...,7
1,From: guykuo@carson.u.washington.edu (Guy Kuo)...,4
2,From: twillis@ec.ecn.purdue.edu (Thomas E Will...,4
3,From: jgreen@amber (Joe Green)\nSubject: Re: W...,1
4,From: jcm@head-cfa.harvard.edu (Jonathan McDow...,14
...,...,...
11309,From: jim.zisfein@factory.com (Jim Zisfein) \n...,13
11310,From: ebodin@pearl.tufts.edu\nSubject: Screen ...,4
11311,From: westes@netcom.com (Will Estes)\nSubject:...,3
11312,From: steve@hcrlgw (Steven Collins)\nSubject: ...,1


In [ ]:
n_l = []
for i in X_data:
    print(i)
    n = len(i)
    n_l.append(n)
max(n_l), min(n_l) #맥스렌민렌

In [ ]:
tk = Tokenizer(num_words=10000 )  #num_words=
tk.fit_on_texts(X_data)
t_x_data = tk.texts_to_matrix(X_data) #매트릭스로꺼내기  입력도 매트릭스로
t_x_data.shape

(11314, 10000)

In [ ]:
type(X_data[0]) # 이게 str으로나오면 매트릭스로 전달해도 문제없다는 뜻

str

In [ ]:
t1_x_data = tk.texts_to_matrix(X_data, mode='count')
t1_x_data

array([[ 0.,  6.,  2., ...,  0.,  0.,  0.],
       [ 0.,  4.,  1., ...,  0.,  0.,  0.],
       [ 0., 21.,  6., ...,  0.,  0.,  0.],
       ...,
       [ 0., 12.,  1., ...,  0.,  0.,  0.],
       [ 0.,  3.,  2., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.]])

In [ ]:
t2_x_data = tk.texts_to_matrix(X_data, mode='tfidf') #tf라는카운트에 가중치를곱
t2_x_data

#문장의벡터화

array([[0.        , 2.03303026, 1.26800369, ..., 0.        , 0.        ,
        0.        ],
       [0.        , 1.73776026, 0.74890341, ..., 0.        , 0.        ,
        0.        ],
       [0.        , 2.94532413, 2.09075818, ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.        , 2.53779767, 0.74890341, ..., 0.        , 0.        ,
        0.        ],
       [0.        , 1.52826285, 1.26800369, ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ]])

In [ ]:
X_data1 = t_x_data
X_data2 = t1_x_data
X_data3 = t2_x_data

In [ ]:
y다중클래스분류 y를 투카테

In [ ]:
Y_data = to_categorical(y_data)
Y_data

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)

In [ ]:
len(X_data1), len(Y_data) ,len(X_data2),len(X_data3)

(11314, 11314, 11314, 11314)

In [ ]:
from sklearn.model_selection import train_test_split
#단어유무특징을갖는data
t_x_1, tt_x_1, t_y_1, tt_y_1 = train_test_split(X_data1, Y_data, random_state=1)
#단어 빈도수(카운트) 특징을갖는data
t_x_2, tt_x_2, t_y_2, tt_y_2 = train_test_split(X_data2, Y_data, random_state=1)
#단어 tfidf 가중치 특징을갖는data
t_x_3, tt_x_3, t_y_3, tt_y_3 = train_test_split(X_data3, Y_data, random_state=1)

In [ ]:
#다중클래스분류소프트맥스 아웃풋20개 인풋10000개

In [ ]:
#모델생성

from keras import Sequential # 모델
from keras.layers import SimpleRNN, LSTM, GRU,   Dense, Embedding   ,Dropout
from tensorflow.keras.optimizers import SGD, RMSprop, Adam

m = Sequential()
m.add( Dense( 256, input_shape=(10000,), activation='relu' ) )

m.add(Dropout(0.3))

m.add( Dense( 128, input_shape=(10000,),   ) )

m.add(Dropout(0.3))

m.add( Dense( 20, input_shape=(10000,), activation='softmax' ) )
m.compile(optimizer = 'adam' , loss='categorical_crossentropy'  , metrics=['accuracy'])  # 옵티마이저 알엠 기본값, 옵티마이저는 백프로파게이션값을 계산하는 함수다. 액티베이션은 인풋-아웃풋 과정에서 웨이트를 갱신하는 함수 # 로스펑션: 은 손실값을 계산하는 함수, 아웃풋기준 이진분류니까 binary, 다중분류면 sparse_categorical_crossentropy 다중분류인데 아웃풋을 원핫했으면 categorical_crossentropy : 아웃풋 원핫은 표현방법일 뿐 이진분류가 아니라 그대로 다중분류다 # mean_squared_error 회귀면 엠엣스이 이것만 기억 m.compile(loss='mse' metrics = ['mae'])  

 

In [ ]:
hy = m.fit( t_x_1, t_y_1,  validation_data=(tt_x_1, tt_y_1 ),epochs=100   )  # 배치사이즈는 2의 배수로 #콜백사용시: 벨리데이션셋 권장 # 핏에 의한 가중치는 m 에 학습되어 있다 hy가 아니다. 딥러닝 모델은 여전히 m 이다. # 추가학습이 필요할 경우 그대로 아래칸에 복붙하면 애초에 에폭을 여러번 돌린 것과 같은 효과를 낼 수 있다.
#데이터를매트릭스로만들엇기떄문에 가능  시퀀셜데이터인데 시퀀셜모델없이 벡터화해서 

Epoch 1/100
266/266 [==============================] - 9s 33ms/step - loss: 0.2112 - accuracy: 0.9472 - val_loss: 0.3820 - val_accuracy: 0.8989
Epoch 2/100
266/266 [==============================] - 10s 37ms/step - loss: 0.0805 - accuracy: 0.9862 - val_loss: 0.3654 - val_accuracy: 0.8968
Epoch 3/100
266/266 [==============================] - 8s 31ms/step - loss: 0.0556 - accuracy: 0.9899 - val_loss: 0.4178 - val_accuracy: 0.8915
Epoch 4/100
266/266 [==============================] - 10s 36ms/step - loss: 0.0432 - accuracy: 0.9941 - val_loss: 0.4180 - val_accuracy: 0.8989
Epoch 5/100
266/266 [==============================] - 10s 36ms/step - loss: 0.0204 - accuracy: 0.9973 - val_loss: 0.4346 - val_accuracy: 0.8964
Epoch 6/100
266/266 [==============================] - 10s 39ms/step - loss: 0.0318 - accuracy: 0.9940 - val_loss: 0.4941 - val_accuracy: 0.8809
Epoch 7/100
266/266 [==============================] - 8s 31ms/step - loss: 0.0406 - accuracy: 0.9912 - val_loss: 0.5969 - val_accur